In [1]:
from utils import *
import glob
sns.set_style('whitegrid')


T=int(1e3)
ts = list(range(1,T+1))
gamma=4

In [2]:
from itertools import product


mps, n_gossips = [f"baseline", f"Flooding-Absorption", f"Flooding", f"Flooding-RandomStop-0.5"], [1, None]
# collect datas in parallel
partial_params = list(product(mps, n_gossips))
# run experiment only once for baseline
partial_params = [item for item in partial_params if
                  item[0] != "baseline" or (item[0] == "baseline" and item[1] is None)]
# remove Hitting+Gossiping
partial_params = [item for item in partial_params if "Absorption" not in item[0] or item[1] is None]
# remove RS+Gossiping
partial_params = [item for item in partial_params if "RandomStop" not in item[0] or item[1] is None]

print(partial_params)

[('baseline', None), ('Flooding-Absorption', None), ('Flooding', 1), ('Flooding', None), ('Flooding-RandomStop-0.5', None)]


In [3]:
## new plots for regret/comm complexity vs. t
# legends=None
legends = ["baseline", "Flooding-A", "gossiping", "Flooding", "Prob Flooding (0.5)", "IRS"]
# for dynamic in ["", "_dynamic_dense", "_dynamic_sparse"]:
for dynamic in ["", "_dynamic_dense"]:
    if dynamic == "":
        RGs = ['ER', 'BA', 'SBM']
    else:
        RGs = ['ER']
    for RG in RGs:
        path = f"results-uniform_N_100_K_50_k_20/heterogeneous_K=50{dynamic}/{RG}"
        additional_path = f"results-uniform_N_100_K_50_k_20/additional_prob_flooding/heterogeneous_K=50{dynamic}/{RG}"
        for mode in ["Regret", "Communication", "Message"]:
            names2 = glob.glob(f"{path}/{mode}_final_t_p=1.0_gamma={gamma}*.npz")
            tmp2 = np.load(names2[0])
            mean, std = tmp2['arr_0'], tmp2['arr_1']
            # if mean.shape[0] > 4:
            #     mean, std = np.delete(mean, 1, 0), np.delete(std, 1, 0)
            
            # overlay results for IRS (Flooding with gamma=1, no gossip)
            names = glob.glob(f"{path}/{mode}_final_t_p=1.0_gamma=1*.npz")
            tmp1 = np.load(names[0])
            mean1, std1 = tmp1['arr_0'], tmp1['arr_1']
            mean = np.concatenate((mean, mean1[-1].reshape((1,T))))
            std = np.concatenate((std, std1[-1].reshape((1,T))))
            
            # # overlay results for probabilistic flooding
            # names = glob.glob(f"{additional_path}/{mode}_final_t_p=1.0_gamma=4*.npz")
            # tmp1 = np.load(names[0])
            # mean1, std1 = tmp1['arr_0'], tmp1['arr_1']
            # mean = np.concatenate((mean, mean1.reshape((2,T))))
            # std = np.concatenate((std, std1.reshape((2,T))))
            # # overlay results for naive stopping
            # regret_naive = np.load(f"{path}/{RG_model_name}_naive/Regret_final_t_p=1.0_gamma=2_{RG_model}.npz")
            # regret_mean_naive, regret_std_naive = regret_naive['arr_0'], regret_naive['arr_1']
        
            title = f"{mode} vs. t ({RG}, gamma={gamma}, {dynamic})"
            fname = f"{path}/{mode}_final_t_plot.pdf"
        
            plot_final(mean, std, ts, title, "t", fname, legends)